In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
from contracts import BaseplateDesign
from code_generation import CodeOutput
from task_output import Task 

/Users/dimbul/Desktop/personal/deep-learning/llm-agentic-flow/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
problem_txt  = "./problem/constraint.txt"
output_txt   = "./problem/output.txt"

with open(problem_txt, 'r') as fp:
    problem_txt  = fp.read()

In [3]:
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI()


### 1. Requriment Parsing and Structuring  

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")
structured_llm = llm.with_structured_output(BaseplateDesign)
output = structured_llm.invoke(problem_txt)

/Users/dimbul/Desktop/personal/deep-learning/llm-agentic-flow/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [5]:
output.dict()


/var/folders/np/wf5kwsmd3v73ryvzxmxbz4m00000gn/T/ipykernel_71836/1946351800.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  output.dict()


{'load_capacity': {'primary_vertical_load': 3000.0,
  'lateral_load': 300.0,
  'factor_of_safety': 2.0},
 'dimensions_and_mounting': {'max_footprint': '200 mm x 200 mm',
  'plate_thickness': 10.0,
  'mounting_holes': 4,
  'hole_spacing': 'Must fit within 200 mm x 200 mm footprint'},
 'deflection_constraint': {'max_deflection': 0.5},
 'weight_and_cost': {'max_weight': 3.0, 'estimated_cost': 20.0},
 'manufacturability': {'fabrication_methods': ['laser cutting',
   'waterjet cutting',
   'drilling'],
  'geometry_simplicity': True},
 'safety_and_installation': {'sharp_edges': False,
  'rounded_corners': True,
  'bolt_hole_reinforcement': True,
  'reference_mark': 'Center hole or marked reference'}}

### 2. Code Geneartion using 5 Softwares and 3 Scripts

In [6]:

prompt_code_generation = [(
        "system",
        "You are specialized in mechanical hardware engineering job. Your Job is to generate 3 distinct Automatable Complete CAD Script using each software package,  1. SolidWorks \n 2. Autodesk \n 3. Openscad \n 4. Freecad \n  5.Onshape \n.  You will be provided with requriments in JSON format. Generate a total of 15 scripts",
    ),
    ("human", f"Here is the string version constraint Data - {str(output.json())}")]

code_msg = llm.invoke(prompt_code_generation)
code_structure = llm.with_structured_output(CodeOutput)
code_output : CodeOutput= code_structure.invoke( code_msg.content)




/var/folders/np/wf5kwsmd3v73ryvzxmxbz4m00000gn/T/ipykernel_71836/4113263486.py:5: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  ("human", f"Here is the string version constraint Data - {str(output.json())}")]


In [ ]:
for code_gen in code_output.code_output:
    print("-----------------------------------------")
    print("software used -- ", code_gen.software_used)
    print(code_gen.code_snippet)

### 3. Generating Design Plan 

In [ ]:

design_plan_generator = [
    (
        "system",
        "You are specialized in mechanical hardware engineering job and you are a manager of building the product.  You are tasked to build the design plan of the component.  You need to detail all the steps in the process.  Priotize the steps and generate a plan",
    ),
    ("human", f"Here is the string version constraint and job details - {str(output.json())}"),
]
design_plan = llm.invoke(design_plan_generator)
design_plan_structure = llm.with_structured_output(Task)
design_plan_output : Task= design_plan_structure.invoke( design_plan.content)



In [ ]:
for task_data in design_plan_output.sub_tasks:
    print("-----------------------------------------------------------------------")
    print("Desc. of the Sub-Task ::", task_data.description)
    print("Actions :: ", ", ".join(task_data.actions))
    print("Acceptance Criteria :: ", ", ".join(task_data.acceptance_criteria))
